# Extension 2: PubMedBERT with Hard Negatives and Focal Loss

## Context: Starting from PubMedBERT Baseline

In the baseline, we achieved **39.30% sentence-level F1** using PubMedBERT with sentence-pair architecture:
- Input: `[CLS] claim [SEP] sentence [SEP]`
- Model: PubMedBERT encoder + evidence head (binary) + claim classifier (3-way)
- Training: All claims including NEI, but evidence loss uses standard BCE

## Problem Identified

While the baseline already processes NEI claims, there are opportunities for improvement:

1. **Evidence Loss Imbalance**: Standard BCE struggles with class imbalance (most sentences are non-evidence)
2. **Inference Rule**: Forcing "no evidence ⇒ NEI" can override correct stance predictions
3. **Negative Quality**: Current approach uses random claim+random doc pairings, which can be noisy

## Proposed Solution

This extension implements three targeted improvements:

1. **Better Negatives**: Use real NEI examples from NEI claims with their cited documents (not random pairings)
2. **Focal Loss**: Replace standard BCE with Focal Loss to better handle evidence class imbalance
3. **Cleaner Inference**: Remove "no evidence ⇒ NEI" forcing, let stance classifier decide
4. **Optional Hard Negatives**: Lexical similarity-based negatives from similar but non-gold documents

## Expected Impact

- **Target**: +2-4% F1 improvement (from 39.30% to 41-43%)
- **Primary benefit**: Better precision through improved evidence loss
- **Secondary benefit**: More accurate NEI classification


In [1]:
!pip install jsonlines

In [2]:
# Install required packages
%pip install -q transformers torch jsonlines tqdm scikit-learn


In [3]:
# Setup: Mount Google Drive and install dependencies
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
import os
import random
import jsonlines
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
from collections import defaultdict, Counter
import numpy as np

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


Mounted at /content/drive
PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4


In [4]:
# Navigate to project directory
# PROJECT_PATH = '/content/drive/MyDrive/cis5300_project'
# Or if cloned from GitHub:
!git clone https://github.com/asxd-10/cis5300_project.git
PROJECT_PATH = '/content/cis5300_project'

if os.path.exists(PROJECT_PATH):
    os.chdir(PROJECT_PATH)
    print(f"Changed to: {os.getcwd()}")
else:
    print(f"Project path not found: {PROJECT_PATH}")
    print("Please update PROJECT_PATH or clone the repository")


Cloning into 'cis5300_project'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (286/286), done.
remote: Compressing objects: 100% (249/249), done.
remote: Total 286 (delta 160), reused 97 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (286/286), 14.30 MiB | 15.77 MiB/s, done.
Resolving deltas: 100% (160/160), done.
Changed to: /content/cis5300_project


In [5]:
# Configuration
MODEL_NAME = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LEN = 256
BATCH_SIZE = 16  # Same as baseline
LEARNING_RATE = 2e-5  # Same as baseline
NUM_EPOCHS = 6  # Same as baseline
EVIDENCE_LOSS_WEIGHT = 2.0  # Weight for evidence loss

# Focal Loss Configuration
FOCAL_ALPHA = 0.75  # Weight for positive class
FOCAL_GAMMA = 2.0   # Focusing parameter

# Hard Negative Mining Configuration
USE_LEXICAL_HARD_NEGATIVES = True  # Enable lexical similarity-based hard negatives
HARD_NEGATIVE_RATIO = 0.3  # Add 30% hard negatives relative to gold examples
MAX_HARD_NEGATIVES_PER_CLAIM = 5  # Max hard negative sentences per claim

RANDOM_SEED = 42

# Set random seeds for reproducibility
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

print(f"Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Device: {DEVICE}")
print(f"  Focal Loss: alpha={FOCAL_ALPHA}, gamma={FOCAL_GAMMA}")
print(f"  Hard Negatives: {USE_LEXICAL_HARD_NEGATIVES}, ratio={HARD_NEGATIVE_RATIO}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Epochs: {NUM_EPOCHS}")


Configuration:
  Model: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext
  Device: cuda
  Focal Loss: alpha=0.75, gamma=2.0
  Hard Negatives: True, ratio=0.3
  Batch Size: 16
  Learning Rate: 2e-05
  Epochs: 6


In [12]:
# Load data
from src.common.data_utils import load_claims, load_corpus
from collections import Counter

train_claims = load_claims('data/scifact/data/claims_train.jsonl')
dev_claims = load_claims('data/scifact/data/claims_dev.jsonl')
corpus = load_corpus('data/scifact/data/corpus.jsonl')

print(f"{len(train_claims)} training claims")
print(f"{len(dev_claims)} dev claims")
print(f"{len(corpus)} documents")

# ============================================================
# EXPLORATORY DATA ANALYSIS
# ============================================================
print("\n" + "="*60)
print("EXPLORATORY DATA ANALYSIS")
print("="*60)

# Analyze data distribution
claims_with_evidence = [c for c in train_claims if c.evidence and c.label]
nei_claims = [c for c in train_claims if not c.evidence or c.label == 'NOT_ENOUGH_INFO']

print(f"\n1. Training Data Breakdown:")
print(f"   Claims WITH evidence: {len(claims_with_evidence)} ({100*len(claims_with_evidence)/len(train_claims):.1f}%)")
print(f"   NOT_ENOUGH_INFO claims: {len(nei_claims)} ({100*len(nei_claims)/len(train_claims):.1f}%)")
print(f"   Total: {len(train_claims)}")

# Label distribution
label_counts = Counter([c.label for c in claims_with_evidence if c.label])
print(f"\n2. Label Distribution (claims with evidence):")
for label, count in label_counts.most_common():
    print(f"   {label}: {count} ({100*count/len(claims_with_evidence):.1f}%)")

# Evidence sentence statistics
evidence_sent_counts = []
for claim in claims_with_evidence:
    total_ev_sents = 0
    for doc_id, ev_list in claim.evidence.items():
        for ev_entry in ev_list:
            total_ev_sents += len(ev_entry.get('sentences', []))
    evidence_sent_counts.append(total_ev_sents)

if evidence_sent_counts:
    print(f"\n3. Evidence Sentence Statistics:")
    print(f"   Mean evidence sentences per claim: {sum(evidence_sent_counts)/len(evidence_sent_counts):.2f}")
    print(f"   Min: {min(evidence_sent_counts)}, Max: {max(evidence_sent_counts)}")
    print(f"   Claims with 1 sentence: {sum(1 for x in evidence_sent_counts if x == 1)}")
    print(f"   Claims with 2+ sentences: {sum(1 for x in evidence_sent_counts if x >= 2)}")

# Document length statistics
doc_lengths = [len(doc.abstract) for doc in corpus.values()]
print(f"\n4. Document Statistics:")
print(f"   Mean sentences per document: {sum(doc_lengths)/len(doc_lengths):.2f}")
print(f"   Min: {min(doc_lengths)}, Max: {max(doc_lengths)}")

# Class imbalance analysis
print(f"\n5. Class Imbalance Analysis:")
print(f"   Evidence sentences per document: ~{sum(evidence_sent_counts)/len(evidence_sent_counts) if evidence_sent_counts else 0:.2f}")
print(f"   Total sentences per document: ~{sum(doc_lengths)/len(doc_lengths):.2f}")
imbalance_ratio = (sum(doc_lengths)/len(doc_lengths)) / (sum(evidence_sent_counts)/len(evidence_sent_counts) if evidence_sent_counts else 1)
print(f"   Imbalance ratio: ~{imbalance_ratio:.1f}:1 (non-evidence : evidence)")
print(f"   This explains why standard BCE loss struggles!")

809 training claims
300 dev claims
5183 documents

EXPLORATORY DATA ANALYSIS

1. Training Data Breakdown:
   Claims WITH evidence: 505 (62.4%)
   NOT_ENOUGH_INFO claims: 304 (37.6%)
   Total: 809

2. Label Distribution (claims with evidence):
   SUPPORT: 332 (65.7%)
   CONTRADICT: 173 (34.3%)

3. Evidence Sentence Statistics:
   Mean evidence sentences per claim: 2.03
   Min: 1, Max: 11
   Claims with 1 sentence: 225
   Claims with 2+ sentences: 280

4. Document Statistics:
   Mean sentences per document: 8.87
   Min: 3, Max: 367

5. Class Imbalance Analysis:
   Evidence sentences per document: ~2.03
   Total sentences per document: ~8.87
   Imbalance ratio: ~4.4:1 (non-evidence : evidence)
   This explains why standard BCE loss struggles!


In [13]:
# Focal Loss for Evidence Classification
class FocalLoss(nn.Module):
    """
    Focal Loss for addressing class imbalance in evidence classification.
    FL(p_t) = -alpha * (1 - p_t)^gamma * log(p_t)
    """
    def __init__(self, alpha=0.75, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.bce = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, logits, targets):
        """
        Args:
            logits: [N] - evidence logits
            targets: [N] - evidence labels (0 or 1)
        """
        # Compute BCE loss
        bce_loss = self.bce(logits, targets.float())

        # Compute p_t (probability of true class)
        probs = torch.sigmoid(logits)
        p_t = probs * targets + (1 - probs) * (1 - targets)

        # Compute focal weight
        focal_weight = (1 - p_t) ** self.gamma

        # Compute alpha_t (class weighting)
        alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)

        # Compute focal loss
        focal_loss = alpha_t * focal_weight * bce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

print("Focal Loss class defined")


Focal Loss class defined


In [14]:
# Lexical similarity filter for hard negatives
def simple_lexical_filter(claim_text, corpus, exclude_doc_ids, max_candidates=10):
    """
    Find documents that share tokens with the claim (simple lexical overlap).
    Returns list of candidate doc_ids that are similar but not in exclude_doc_ids.
    """
    # Simple tokenization (lowercase, split)
    claim_tokens = set(claim_text.lower().split())
    # Remove very common words (stopwords-like)
    stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were'}
    claim_tokens = claim_tokens - stopwords

    if len(claim_tokens) == 0:
        return []

    candidates = []
    for doc_id, doc in corpus.items():
        if doc_id in exclude_doc_ids:
            continue

        # Tokenize document (title + abstract)
        doc_text = (doc.title + " " + " ".join(doc.abstract)).lower()
        doc_tokens = set(doc_text.split())

        # Check overlap
        overlap = claim_tokens & doc_tokens
        if len(overlap) > 0:
            candidates.append(doc_id)
            if len(candidates) >= max_candidates:
                break

    return candidates

print("Lexical filter function defined")


Lexical filter function defined


In [15]:
# Enhanced Dataset with Better Negatives
class ImprovedSciFactSentencePairDataset(Dataset):
    """
    Sentence-pair dataset with improved negative mining:
    1. Real NEI examples from NEI claims with their cited documents (not random pairings)
    2. Optional lexical hard negatives from similar but non-gold documents
    3. All positive examples from gold documents (same as baseline)
    """
    def __init__(self, claims, corpus, tokenizer, max_len=256, mode='train',
                 use_lexical_hard_negatives=True, hard_negative_ratio=0.3, max_hard_negatives_per_claim=5):
        self.examples = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode = mode
        self.label_map = {'SUPPORT': 0, 'CONTRADICT': 1, 'NOT_ENOUGH_INFO': 2}
        self.use_lexical_hard_negatives = use_lexical_hard_negatives and (mode == 'train')
        self.hard_negative_ratio = hard_negative_ratio
        self.max_hard_negatives_per_claim = max_hard_negatives_per_claim

        # Separate claims
        claims_with_evidence = []
        nei_claims = []

        for claim in claims:
            if claim.evidence and len(claim.evidence) > 0:
                claims_with_evidence.append(claim)
            elif claim.label == 'NOT_ENOUGH_INFO':
                nei_claims.append(claim)

        print(f"  Claims with evidence: {len(claims_with_evidence)}")
        print(f"  NOT_ENOUGH_INFO claims: {len(nei_claims)}")

        num_positive = 0
        num_local_negative = 0
        num_nei_negative = 0
        num_hard_negative = 0

        # 1. Positive examples + local negatives from gold documents
        for claim in claims_with_evidence:
            gold_doc_ids = set()

            for doc_id in claim.cited_doc_ids:
                doc_int = int(doc_id)
                if doc_int not in corpus:
                    continue
                doc = corpus[doc_int]
                gold_doc_ids.add(doc_int)

                for sent_idx, sent in enumerate(doc.abstract):
                    # Check if this sentence is evidence
                    is_evidence = 0
                    evidence_label = 'NOT_ENOUGH_INFO'
                    if claim.evidence and str(doc.doc_id) in claim.evidence:
                        for ev in claim.evidence[str(doc.doc_id)]:
                            if sent_idx in ev.get('sentences', []):
                                is_evidence = 1
                                evidence_label = ev.get('label')
                                break

                    claim_label = self.label_map.get(claim.label, 2)

                    self.examples.append({
                        'claim_id': claim.id,
                        'doc_id': doc.doc_id,
                        'sent_idx': sent_idx,
                        'claim': claim.claim,
                        'sentence': sent,
                        'is_evidence': is_evidence,
                        'claim_label': claim_label,
                        'evidence_label_str': evidence_label
                    })

                    if is_evidence:
                        num_positive += 1
                    else:
                        num_local_negative += 1

            # 2. Add lexical hard negatives for this claim
            if self.use_lexical_hard_negatives:
                # Count gold sentences for this claim
                num_gold_sents = sum(len(corpus[int(d)].abstract) for d in claim.cited_doc_ids if int(d) in corpus)
                num_hard_needed = min(int(num_gold_sents * self.hard_negative_ratio), self.max_hard_negatives_per_claim)

                if num_hard_needed > 0:
                    # Find similar documents
                    candidate_docs = simple_lexical_filter(claim.claim, corpus, gold_doc_ids, max_candidates=20)

                    if candidate_docs:
                        # Sample one candidate document
                        selected_doc_id = random.choice(candidate_docs)
                        selected_doc = corpus[selected_doc_id]

                        # Sample sentences from this document
                        num_sents_to_sample = min(num_hard_needed, len(selected_doc.abstract))
                        sent_indices = random.sample(range(len(selected_doc.abstract)), num_sents_to_sample)

                        for sent_idx in sent_indices:
                            self.examples.append({
                                'claim_id': claim.id,
                                'doc_id': selected_doc.doc_id,
                                'sent_idx': sent_idx,
                                'claim': claim.claim,
                                'sentence': selected_doc.abstract[sent_idx],
                                'is_evidence': 0,  # Hard negative: no evidence
                                'claim_label': 2,  # NOT_ENOUGH_INFO
                                'evidence_label_str': 'NOT_ENOUGH_INFO'
                            })
                            num_hard_negative += 1

        # 3. Real NEI examples: NEI claims with their cited documents
        for claim in nei_claims:
            if not claim.cited_doc_ids:
                # If no cited docs, optionally sample random docs (but limit this)
                if mode == 'train' and random.random() < 0.3:  # Only 30% of NEI claims without cited docs
                    available_docs = [d for d in corpus.keys()]
                    if available_docs:
                        sampled_doc_id = random.choice(available_docs)
                        sampled_doc = corpus[sampled_doc_id]
                        # Sample a few sentences
                        num_sents = min(3, len(sampled_doc.abstract))
                        for sent_idx in range(num_sents):
                            self.examples.append({
                                'claim_id': claim.id,
                                'doc_id': sampled_doc.doc_id,
                                'sent_idx': sent_idx,
                                'claim': claim.claim,
                                'sentence': sampled_doc.abstract[sent_idx],
                                'is_evidence': 0,
                                'claim_label': 2,
                                'evidence_label_str': 'NOT_ENOUGH_INFO'
                            })
                            num_nei_negative += 1
                continue

            # Process cited documents for NEI claims (REAL NEI examples, not random)
            for doc_id in claim.cited_doc_ids:
                doc_int = int(doc_id)
                if doc_int not in corpus:
                    continue
                doc = corpus[doc_int]

                # All sentences from NEI claims are non-evidence
                for sent_idx, sent in enumerate(doc.abstract):
                    self.examples.append({
                        'claim_id': claim.id,
                        'doc_id': doc.doc_id,
                        'sent_idx': sent_idx,
                        'claim': claim.claim,
                        'sentence': sent,
                        'is_evidence': 0,  # All non-evidence for NEI claims
                        'claim_label': 2,  # NOT_ENOUGH_INFO
                        'evidence_label_str': 'NOT_ENOUGH_INFO'
                    })
                    num_nei_negative += 1

        print(f"  Total examples: {len(self.examples)}")
        print(f"    Positive (evidence=1): {num_positive}")
        print(f"    Local negatives (non-evidence in gold docs): {num_local_negative}")
        print(f"    NEI negatives (from NEI claims with cited docs): {num_nei_negative}")
        print(f"    Hard negatives (lexical similarity): {num_hard_negative}")

        # Print evidence distribution
        evidence_counts = Counter([ex['is_evidence'] for ex in self.examples])
        print(f"  Evidence distribution: {dict(evidence_counts)}")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        # Tokenize as pair: [CLS] claim [SEP] sentence [SEP]
        encoding = self.tokenizer(
            ex['claim'], ex['sentence'],
            truncation='only_second',  # Truncate sentence, not claim
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        item = {k: v.squeeze(0) for k, v in encoding.items()}
        item['is_evidence'] = torch.tensor(ex['is_evidence'], dtype=torch.float)
        item['claim_label'] = torch.tensor(ex['claim_label'], dtype=torch.long)
        item['claim_id'] = ex['claim_id']
        item['doc_id'] = ex['doc_id']
        item['sent_idx'] = ex['sent_idx']
        return item

print("Improved dataset class defined")

Improved dataset class defined


In [16]:
# Analyze training data distribution
train_labels = [c.label for c in train_claims]
train_has_evidence = [len(c.evidence) > 0 for c in train_claims]

label_counts = Counter(train_labels)
evidence_counts = Counter(train_has_evidence)

print("Training Data Distribution:")
print(f"  Labels: {dict(label_counts)}")
print(f"  Has Evidence: {dict(evidence_counts)}")
print(f"\n  NOT_ENOUGH_INFO claims: {label_counts.get('NOT_ENOUGH_INFO', 0)}")
print(f"  Claims with evidence: {evidence_counts.get(True, 0)}")
print(f"  Claims without evidence: {evidence_counts.get(False, 0)}")


Training Data Distribution:
  Labels: {'NOT_ENOUGH_INFO': 304, 'CONTRADICT': 173, 'SUPPORT': 332}
  Has Evidence: {False: 304, True: 505}

  NOT_ENOUGH_INFO claims: 304
  Claims with evidence: 505
  Claims without evidence: 304


In [17]:
# EXTENSION 1: Enhanced Dataset with Hard Negative Mining
class SciFactSentencePairDatasetWithNegatives(Dataset):
    """
    Sentence-pair dataset with hard negative mining.
    Creates (claim, sentence) examples for sentences from cited docs.
    Adds negative examples from NOT_ENOUGH_INFO claims paired with random documents.
    """
    def __init__(self, claims, corpus, tokenizer, max_len=256, mode='train', negative_ratio=0.5):
        self.examples = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode = mode
        self.label_map = {'SUPPORT': 0, 'CONTRADICT': 1, 'NOT_ENOUGH_INFO': 2}

        # Separate claims with and without evidence
        claims_with_evidence = []
        nei_claims = []

        for claim in claims:
            if claim.evidence and len(claim.evidence) > 0:
                claims_with_evidence.append(claim)
            elif claim.label == 'NOT_ENOUGH_INFO':
                nei_claims.append(claim)

        print(f"  Claims with evidence: {len(claims_with_evidence)}")
        print(f"  NOT_ENOUGH_INFO claims: {len(nei_claims)}")

        # Process claims with evidence (positive examples)
        for claim in claims_with_evidence:
            for doc_id in claim.cited_doc_ids:
                doc_int = int(doc_id)
                if doc_int not in corpus:
                    continue
                doc = corpus[doc_int]
                for sent_idx, sent in enumerate(doc.abstract):
                    # Ground truth: is this sentence evidence for claim?
                    is_evidence = 0
                    evidence_label = 'NOT_ENOUGH_INFO'
                    if claim.evidence and str(doc.doc_id) in claim.evidence:
                        for ev in claim.evidence[str(doc.doc_id)]:
                            if sent_idx in ev.get('sentences', []):
                                is_evidence = 1
                                evidence_label = ev.get('label')
                                break

                    claim_label = self.label_map.get(claim.label, 2)

                    self.examples.append({
                        'claim_id': claim.id,
                        'doc_id': doc.doc_id,
                        'sent_idx': sent_idx,
                        'claim': claim.claim,
                        'sentence': sent,
                        'is_evidence': is_evidence,
                        'claim_label': claim_label,
                        'evidence_label_str': evidence_label
                    })

        num_positive = len(self.examples)
        print(f"  Positive examples created: {num_positive}")

        # EXTENSION: Add hard negative examples (only in training mode)
        if mode == 'train' and negative_ratio > 0:
            num_negatives_needed = int(num_positive * negative_ratio)
            print(f"  Adding {num_negatives_needed} hard negative examples...")

            # Get all document IDs for random sampling
            all_doc_ids = list(corpus.keys())

            # Sample NOT_ENOUGH_INFO claims for negative examples
            sampled_nei_claims = random.sample(nei_claims, min(len(nei_claims), num_negatives_needed))

            for claim in sampled_nei_claims:
                # Pair with a random document (not from cited_doc_ids)
                # Try to find a document that's NOT in cited_doc_ids
                available_docs = [d for d in all_doc_ids if d not in [int(x) for x in claim.cited_doc_ids]]
                if not available_docs:
                    available_docs = all_doc_ids  # Fallback to all docs

                random_doc_id = random.choice(available_docs)
                doc = corpus[random_doc_id]

                # Sample 1-3 random sentences from this document
                num_sents_to_sample = min(random.randint(1, 3), len(doc.abstract))
                sampled_sent_indices = random.sample(range(len(doc.abstract)), num_sents_to_sample)

                for sent_idx in sampled_sent_indices:
                    sent = doc.abstract[sent_idx]

                    # This is a negative example: is_evidence = 0
                    self.examples.append({
                        'claim_id': claim.id,
                        'doc_id': doc.doc_id,
                        'sent_idx': sent_idx,
                        'claim': claim.claim,
                        'sentence': sent,
                        'is_evidence': 0,  # Hard negative: no evidence
                        'claim_label': 2,  # NOT_ENOUGH_INFO
                        'evidence_label_str': 'NOT_ENOUGH_INFO'
                    })

            print(f"  Total examples (pos + neg): {len(self.examples)}")
            print(f"  Negative examples added: {len(self.examples) - num_positive}")

        # Print statistics
        evidence_counts = Counter([ex['is_evidence'] for ex in self.examples])
        print(f"  Evidence distribution: {dict(evidence_counts)}")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        # Tokenize as pair: [CLS] claim [SEP] sentence [SEP]
        encoding = self.tokenizer(
            ex['claim'], ex['sentence'],
            truncation='only_second',  # Truncate sentence, not claim
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        item = {k: v.squeeze(0) for k, v in encoding.items()}
        item['is_evidence'] = torch.tensor(ex['is_evidence'], dtype=torch.float)
        item['claim_label'] = torch.tensor(ex['claim_label'], dtype=torch.long)
        item['claim_id'] = ex['claim_id']
        item['doc_id'] = ex['doc_id']
        item['sent_idx'] = ex['sent_idx']
        return item

print("Dataset class defined with hard negative mining!")


Dataset class defined with hard negative mining!


In [18]:
# Load tokenizer and create datasets
print(f"Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print("\nCreating training dataset with hard negatives...")
train_dataset = ImprovedSciFactSentencePairDataset(
    train_claims, corpus, tokenizer, max_len=MAX_LEN, mode='train',
    use_lexical_hard_negatives=USE_LEXICAL_HARD_NEGATIVES,
    hard_negative_ratio=HARD_NEGATIVE_RATIO,
    max_hard_negatives_per_claim=MAX_HARD_NEGATIVES_PER_CLAIM
)
print("\nCreating dev dataset (no negatives)...")

dev_dataset = ImprovedSciFactSentencePairDataset(
    dev_claims, corpus, tokenizer, max_len=MAX_LEN, mode='dev',
    use_lexical_hard_negatives=False  # No hard negatives in dev
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"\nData loaders created:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Dev batches: {len(dev_loader)}")


Loading tokenizer: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]


Creating training dataset with hard negatives...
  Claims with evidence: 505
  NOT_ENOUGH_INFO claims: 304
  Total examples: 9801
    Positive (evidence=1): 1025
    Local negatives (non-evidence in gold docs): 4660
    NEI negatives (from NEI claims with cited docs): 2741
    Hard negatives (lexical similarity): 1375
  Evidence distribution: {0: 8776, 1: 1025}

Creating dev dataset (no negatives)...
  Claims with evidence: 188
  NOT_ENOUGH_INFO claims: 112
  Total examples: 3121
    Positive (evidence=1): 366
    Local negatives (non-evidence in gold docs): 1789
    NEI negatives (from NEI claims with cited docs): 966
    Hard negatives (lexical similarity): 0
  Evidence distribution: {0: 2755, 1: 366}

Data loaders created:
  Train batches: 613
  Dev batches: 196


In [19]:
# Model: PubMedBERT encoder + two heads (same as original)
class PubMedBERT_SciFact(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden = self.encoder.config.hidden_size
        # Evidence classifier (binary) on pooled output
        self.evidence_head = nn.Linear(hidden, 1)
        # Claim-level classifier (3-way) -- aggregated per claim
        self.claim_classifier = nn.Linear(hidden, 3)

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        pooled = outputs.last_hidden_state[:, 0, :]  # CLS token
        evidence_logit = self.evidence_head(pooled).squeeze(-1)  # [batch]
        claim_logit = self.claim_classifier(pooled)  # [batch, 3]
        return evidence_logit, claim_logit

model = PubMedBERT_SciFact(MODEL_NAME).to(DEVICE)
print(f"Model instantiated. Hidden size: {model.encoder.config.hidden_size}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Model instantiated. Hidden size: 768
Total parameters: 109,485,316


In [20]:
# Loss functions and optimizer
# Loss functions
focal_loss_evidence = FocalLoss(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA).to(DEVICE)
loss_fn_claim = nn.CrossEntropyLoss()
loss_fn_claim = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

print("Optimizer configured")


Optimizer configured


In [21]:
# Training loop with hard negative examples
def train_one_epoch():
    model.train()
    running_loss = 0.0
    running_ev_loss = 0.0
    running_claim_loss = 0.0

    evidence_correct = 0
    evidence_total = 0
    claim_correct = 0
    claim_total = 0

    for batch in tqdm(train_loader, desc='Train'):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        evidence_labels = batch['is_evidence'].to(DEVICE)
        claim_labels = batch['claim_label'].to(DEVICE)

        optimizer.zero_grad()
        ev_logits, claim_logits = model(input_ids, attention_mask)

        # Evidence loss (per sentence)
        # Evidence loss: Use Focal Loss
        loss_ev = focal_loss_evidence(ev_logits, evidence_labels)

        # Claim loss: aggregate sentence-level logits per claim
        claim_ids = batch['claim_id']
        claim_to_idxs = defaultdict(list)
        for i, cid in enumerate(claim_ids):
            claim_to_idxs[cid].append(i)

        # Average claim_logits across sentences for same claim
        agg_claim_logits = []
        agg_claim_labels = []
        for cid, idxs in claim_to_idxs.items():
            logits_avg = claim_logits[idxs].mean(dim=0, keepdim=True)  # [1,3]
            agg_claim_logits.append(logits_avg)
            agg_claim_labels.append(claim_labels[idxs[0]])

        if agg_claim_logits:
            agg_claim_logits = torch.cat(agg_claim_logits, dim=0).to(DEVICE)
            agg_claim_labels = torch.tensor(agg_claim_labels, dtype=torch.long).to(DEVICE)
            loss_claim = loss_fn_claim(agg_claim_logits, agg_claim_labels)
        else:
            loss_claim = torch.tensor(0.0, device=DEVICE)

        # Combined loss (FIXED: weight evidence loss, not claim loss)
        total_loss = loss_claim + EVIDENCE_LOSS_WEIGHT * loss_ev

        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Statistics
        running_loss += total_loss.item()
        running_ev_loss += loss_ev.item()
        running_claim_loss += loss_claim.item()

        # Evidence accuracy
        ev_preds = (torch.sigmoid(ev_logits) > 0.5).float()
        evidence_correct += (ev_preds == evidence_labels).sum().item()
        evidence_total += evidence_labels.size(0)

        # Claim accuracy
        if agg_claim_logits is not None and len(agg_claim_logits) > 0:
            claim_preds = agg_claim_logits.argmax(dim=1)
            claim_correct += (claim_preds == agg_claim_labels).sum().item()
            claim_total += len(agg_claim_labels)

    avg_loss = running_loss / len(train_loader)
    avg_ev_loss = running_ev_loss / len(train_loader)
    avg_claim_loss = running_claim_loss / len(train_loader)
    ev_acc = evidence_correct / evidence_total if evidence_total > 0 else 0
    claim_acc = claim_correct / claim_total if claim_total > 0 else 0

    return avg_loss, avg_ev_loss, avg_claim_loss, ev_acc, claim_acc

print("Training function defined")


Training function defined


In [23]:
NEGATIVE_RATIO=0.3

In [24]:
# Training
print("TRAINING WITH HARD NEGATIVE MINING")
print(f"Negative ratio: {NEGATIVE_RATIO}")
print(f"Epochs: {NUM_EPOCHS}")
print("="*60)

best_loss = float('inf')
training_history = []

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")

    avg_loss, avg_ev_loss, avg_claim_loss, ev_acc, claim_acc = train_one_epoch()

    training_history.append({
        'epoch': epoch + 1,
        'loss': avg_loss,
        'ev_loss': avg_ev_loss,
        'claim_loss': avg_claim_loss,
        'ev_acc': ev_acc,
        'claim_acc': claim_acc
    })

    print(f"  Loss: {avg_loss:.4f} (ev: {avg_ev_loss:.4f}, claim: {avg_claim_loss:.4f})")
    print(f"  Evidence Acc: {ev_acc:.4f}")
    print(f"  Claim Acc: {claim_acc:.4f}")

    # Save checkpoint
    checkpoint_path = f'models/claim_verifier/pubmedbert_ext1_epoch{epoch+1}.pt'
    os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
    torch.save(model.state_dict(), checkpoint_path)
    print(f"  Saved: {checkpoint_path}")

    if avg_loss < best_loss:
        best_loss = avg_loss
        best_checkpoint = checkpoint_path

print(f"\nTraining complete! Best loss: {best_loss:.4f}")
print(f"Best checkpoint: {best_checkpoint}")


TRAINING WITH HARD NEGATIVE MINING
Negative ratio: 0.3
Epochs: 6

Epoch 1/6


Train: 100%|██████████| 613/613 [07:15<00:00,  1.41it/s]


  Loss: 0.6132 (ev: 0.0331, claim: 0.5469)
  Evidence Acc: 0.8639
  Claim Acc: 0.7570
  Saved: models/claim_verifier/pubmedbert_ext1_epoch1.pt

Epoch 2/6


Train: 100%|██████████| 613/613 [07:16<00:00,  1.41it/s]


  Loss: 0.1627 (ev: 0.0260, claim: 0.1108)
  Evidence Acc: 0.8958
  Claim Acc: 0.9650
  Saved: models/claim_verifier/pubmedbert_ext1_epoch2.pt

Epoch 3/6


...

  Loss: 0.0638 (ev: 0.0160, claim: 0.0318)
  Evidence Acc: 0.9496
  Claim Acc: 0.9926
  Saved: models/claim_verifier/pubmedbert_ext1_epoch4.pt

Epoch 5/6


Train: 100%|██████████| 613/613 [07:16<00:00,  1.40it/s]


  Loss: 0.0412 (ev: 0.0106, claim: 0.0200)
  Evidence Acc: 0.9705
  Claim Acc: 0.9944
  Saved: models/claim_verifier/pubmedbert_ext1_epoch5.pt

Epoch 6/6


Train: 100%|██████████| 613/613 [07:16<00:00,  1.40it/s]


  Loss: 0.0258 (ev: 0.0066, claim: 0.0127)
  Evidence Acc: 0.9833
  Claim Acc: 0.9961
  Saved: models/claim_verifier/pubmedbert_ext1_epoch6.pt

Training complete! Best loss: 0.0258
Best checkpoint: models/claim_verifier/pubmedbert_ext1_epoch6.pt


In [26]:
# Generate predictions for evaluation
def generate_predictions(model, claims, corpus, tokenizer, device, threshold=0.5):
    """
    Generate predictions in SciFact format.
    Uses oracle retrieval (gold documents) for now.
    """
    model.eval()
    predictions = []

    with torch.no_grad():
        for claim in tqdm(claims, desc="Generating predictions"):
            if not claim.cited_doc_ids:
                predictions.append({
                    'id': claim.id,
                    'label': 'NOT_ENOUGH_INFO',
                    'evidence': {}
                })
                continue

            # Use oracle retrieval (first cited doc)
            doc_id = int(claim.cited_doc_ids[0])
            if doc_id not in corpus:
                predictions.append({
                    'id': claim.id,
                    'label': 'NOT_ENOUGH_INFO',
                    'evidence': {}
                })
                continue

            doc = corpus[doc_id]
            claim_evidence = {}

            # Process each sentence in the document
            sentence_scores = []
            for sent_idx, sent in enumerate(doc.abstract):
                # Tokenize claim-sentence pair
                encoding = tokenizer(
                    claim.claim, sent,
                    truncation='only_second',
                    padding='max_length',
                    max_length=MAX_LEN,
                    return_tensors='pt'
                ).to(device)

                # Get predictions
                ev_logits, claim_logits = model(encoding['input_ids'], encoding['attention_mask'])
                ev_prob = torch.sigmoid(ev_logits).item()

                sentence_scores.append({
                    'sent_idx': sent_idx,
                    'ev_prob': ev_prob,
                    'claim_logits': claim_logits[0].cpu().numpy()
                })

            # Aggregate claim-level prediction (average logits)
            if sentence_scores:
                avg_claim_logits = np.mean([s['claim_logits'] for s in sentence_scores], axis=0)
                pred_label_idx = np.argmax(avg_claim_logits)
                label_map = {0: 'SUPPORT', 1: 'CONTRADICT', 2: 'NOT_ENOUGH_INFO'}
                pred_label = label_map[pred_label_idx]
            else:
                pred_label = 'NOT_ENOUGH_INFO'

            # Select evidence sentences above threshold
            pred_evidence_sents = [
                s['sent_idx'] for s in sentence_scores
                if s['ev_prob'] > threshold
            ]

            # Build prediction
            prediction = {
                'id': claim.id,
                'label': pred_label,
                'evidence': {}
            }

            # Dont force NEI - use stance classifier's prediction
            if pred_evidence_sents:
                prediction['evidence'][str(doc_id)] = [{
                    'sentences': pred_evidence_sents,
                    'label': pred_label
                }]
            # If no evidence but classifier says SUPPORT/CONTRADICT, still use that label
            # (pred_label already set from classifier above)

            predictions.append(prediction)

    return predictions

print("Prediction function defined")


Prediction function defined


In [ ]:
# Evaluate on dev set with different thresholds
print("EVALUATION: Testing Different Thresholds")

best_f1 = 0
best_threshold = 0.5
results = []

for threshold in [0.3, 0.4, 0.5, 0.55, 0.6, 0.7]:
    print(f"\n--- Threshold: {threshold} ---")

    predictions = generate_predictions(model, dev_claims, corpus, tokenizer, DEVICE, threshold=threshold)

    # Save predictions
    output_path = f'output/dev/pubmedbert_ext1_thresh{int(threshold*100)}.jsonl'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with jsonlines.open(output_path, 'w') as writer:
        writer.write_all(predictions)

    # Evaluate using the scoring script
    import subprocess
    result = subprocess.run(
        ['python', 'src/evaluation/score_claims.py',
         '--gold', 'data/scifact/data/claims_dev.jsonl',
         '--predictions', output_path],
        capture_output=True,
        text=True
    )

    print(result.stdout)

    # Parse F1 from output (simple extraction)
    if 'Sentence-level' in result.stdout:
        lines = result.stdout.split('\n')
        # Find the line with "Sentence-level" and look for F1 in subsequent lines
        for i, line in enumerate(lines):
            if 'Sentence-level' in line:
                # Look for F1 in the next 5 lines
                for j in range(i+1, min(i+6, len(lines))):
                    if 'F1:' in lines[j]:
                        try:
                            f1 = float(lines[j].split('F1:')[1].strip().split()[0])
                            results.append({'threshold': threshold, 'f1': f1})
                            if f1 > best_f1:
                                best_f1 = f1
                                best_threshold = threshold
                            break
                        except:
                            pass
                break

print(f"Best Threshold: {best_threshold} (F1: {best_f1:.4f})")

EVALUATION: Testing Different Thresholds

--- Threshold: 0.3 ---


Generating predictions: 100%|██████████| 300/300 [00:50<00:00,  5.90it/s]


CLAIM VERIFICATION EVALUATION

Loading data...
  Gold claims: 300
  Predictions: 300

Computing metrics...

RESULTS

Abstract-level (Retrieval):
  Precision: 1.0000
  Recall:    0.4749
  F1:        0.6440

Sentence-level (Evidence + Label): PRIMARY METRIC
  Precision: 0.2855
  Recall:    0.4290
  F1:        0.3428

Label-only:
  Accuracy:  0.0000

Interpretation:
  Retrieval is working reasonably
  Evidence extraction is competitive


--- Threshold: 0.4 ---


Generating predictions: 100%|██████████| 300/300 [00:48<00:00,  6.19it/s]


CLAIM VERIFICATION EVALUATION

Loading data...
  Gold claims: 300
  Predictions: 300

Computing metrics...

RESULTS

Abstract-level (Retrieval):
  Precision: 1.0000
  Recall:    0.4484
  F1:        0.6191

Sentence-level (Evidence + Label): PRIMARY METRIC
  Precision: 0.2970
  Recall:    0.4016
  F1:        0.3415

Label-only:
  Accuracy:  0.0000

Interpretation:
  Retrieval is working reasonably
  Evidence extraction is competitive


--- Threshold: 0.5 ---


...

CLAIM VERIFICATION EVALUATION

Loading data...
  Gold claims: 300
  Predictions: 300

Computing metrics...

RESULTS

Abstract-level (Retrieval):
  Precision: 1.0000
  Recall:    0.4130
  F1:        0.5846

Sentence-level (Evidence + Label): PRIMARY METRIC
  Precision: 0.3090
  Recall:    0.3579
  F1:        0.3316

Label-only:
  Accuracy:  0.0000

Interpretation:
  Retrieval is working reasonably
  Evidence extraction is competitive


--- Threshold: 0.6 ---


Generating predictions: 100%|██████████| 300/300 [00:49<00:00,  6.12it/s]


CLAIM VERIFICATION EVALUATION

Loading data...
  Gold claims: 300
  Predictions: 300

Computing metrics...

RESULTS

Abstract-level (Retrieval):
  Precision: 1.0000
  Recall:    0.4071
  F1:        0.5786

Sentence-level (Evidence + Label): PRIMARY METRIC
  Precision: 0.3098
  Recall:    0.3470
  F1:        0.3273

Label-only:
  Accuracy:  0.0000

Interpretation:
  Retrieval is working reasonably
  Evidence extraction is competitive


--- Threshold: 0.7 ---


Generating predictions: 100%|██████████| 300/300 [00:49<00:00,  6.12it/s]

CLAIM VERIFICATION EVALUATION

Loading data...
  Gold claims: 300
  Predictions: 300

Computing metrics...

RESULTS

Abstract-level (Retrieval):
  Precision: 1.0000
  Recall:    0.3658
  F1:        0.5356

Sentence-level (Evidence + Label): PRIMARY METRIC
  Precision: 0.3175
  Recall:    0.3115
  F1:        0.3145

Label-only:
  Accuracy:  0.0000

Interpretation:
  Retrieval is working reasonably
  Evidence extraction is competitive

Best Threshold: 0.5 (F1: 0.0000)


## Results Summary

Compare with baseline:
- **Baseline (PubMedBERT)**: 39.30% F1
- **With Hard Negatives + Focal Loss**: 34.28% F1 (best at threshold 0.3)

### Analysis

This extension did not improve upon the baseline. While Focal Loss and better negative mining improved NEI classification, they made the evidence head more conservative, reducing recall on evidence sentences. The combination of increased NEI training examples and Focal Loss's focus on hard examples led the model to be overly cautious about predicting evidence, which hurt sentence-level F1. This demonstrates that the baseline's simpler approach (standard BCE with all claims) was already well-tuned for the sentence-level F1 metric, which primarily rewards correct evidence extraction for SUPPORT/CONTRADICT claims rather than NEI classification accuracy.
